In [29]:
import tensorflow as tf
import numpy as np
from music21 import *
import os
import json
import numpy as np
import pickle
from random import shuffle
import zipfile
import gensim
import random
from datetime import datetime
import shutil
#may not need metrics

In [30]:
def build_discriminator(music,num_neurons, num_dense_neurons):
    #Note: LSTM not gpu optimized
    bidir = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(num_neurons), input_shape=(3072, 50))
    hidden_dense = tf.keras.layers.Dense(num_dense_neurons, activation = tf.nn.relu)
    condense = tf.keras.layers.Dense(1)
    sigmoid = tf.keras.layers.Activation('sigmoid')
    
    layer1 = bidir(music)
    layer2 = hidden_dense(layer1)
    layer3 = condense(layer2)
    final_output = sigmoid(layer3)
    return tf.keras.Model(music, final_output)

In [31]:
def build_generator(original_input_music, input_music, num_neurons):
        hidden_output = tf.keras.layers.LSTM(200, activation = tf.math.sigmoid, input_shape=(1536,50))(input_music)
        repeated_output = tf.keras.layers.RepeatVector(1536)(hidden_output)
        end_sequence = tf.keras.layers.LSTM(50, return_sequences = True, activation = tf.math.sigmoid)(repeated_output)
        results = tf.keras.layers.Concatenate(1)([input_music, end_sequence])
        return tf.keras.Model(original_input_music, results), results
    

In [32]:
def next_batch(lis, length):
    counter = 0
    string = ""
    for item in lis:
        string = str(item)
        zipfilePath = ("./clean_data/" + string + ".zip")
        zip = zipfile.ZipFile(zipfilePath)
        zip.extractall("./clean_data")
        pickle_file = open("./clean_data/clean_data/" + string + ".pickle","rb")
        lis = pickle.load(pickle_file)
        pickle_file.close()
        ray = np.array(lis)
        
        ray = np.transpose(ray)
        index = random.randint(0,ray.shape[0]-1-length)
        ray = ray[index:index+length, :]
        if counter == 0:
            #print("made final")
            final = np.reshape(ray, [1,length,88])
        else:
            #print("catted")
            final = np.concatenate((final, np.reshape(ray, [1,length,88])), 0)
            #print(final.shape)
        #os.remove("./clean_data/" + string + ".pickle")
        shutil.rmtree("./clean_data/clean_data")
        counter = counter + 1
    return (final)

In [33]:
music = tf.keras.Input([3072,88])
input_music = tf.keras.Input([1536,88])

coded_input_music = tf.keras.Input([3072,50])

test_discrim_input = tf.keras.Input([3072,50])

encode = tf.keras.layers.Dense(50, activation = "relu")
decode = tf.keras.layers.Dense(88, activation = 'sigmoid')

encoded_dis_music = tf.keras.layers.TimeDistributed(encode)(music)
encoded_gen_music_input = tf.keras.layers.TimeDistributed(encode)(input_music)

decoded_music = tf.keras.layers.TimeDistributed(decode)(encoded_dis_music)
decoded_input_music = tf.keras.layers.TimeDistributed(decode)(coded_input_music)

decoder_model = tf.keras.Model(coded_input_music, decoded_input_music)

test_decoded_model = tf.keras.Model(music, encoded_dis_music)

decoded_input_music = tf.keras.layers.TimeDistributed(decode)(encoded_gen_music_input)

encoder_decoder = tf.keras.Model(inputs = [music, input_music], outputs = [decoded_music, decoded_input_music])

adam = tf.keras.optimizers.Adam(learning_rate = .01)
encoder_decoder.compile(optimizer = 'adam' , loss='categorical_crossentropy', loss_weights = [.5,.5])

encode.trainable = False
decode.trainable = False

In [34]:

discriminator = build_discriminator(test_discrim_input, 128, 50)

real_discriminator_output = discriminator(encoded_dis_music)

real_discriminator = tf.keras.Model(music, real_discriminator_output)

discriminator.compile(optimizer = 'adam', loss='binary_crossentropy')
real_discriminator.compile(optimizer = 'adam', loss='binary_crossentropy')

In [35]:
generator, to_dis = build_generator(input_music, encoded_gen_music_input,128)
discriminator.trainable = False
real_discriminator.trainable = False
fake_lables = discriminator(to_dis)
combined = tf.keras.Model(input_music, fake_lables)
combined.compile(optimizer = 'adam', loss='binary_crossentropy')

In [38]:
num_total_data = 15509
batch_size = 100
epochs = 1

In [39]:
for e in range(epochs):
    num_batches = num_total_data // batch_size
    dis_list = list(range(num_total_data))
    gen_list = list(range(num_total_data))
    random.shuffle(dis_list)
    random.shuffle(gen_list)
    for i in range(num_batches):
        print("getting data")
        start_data_gen = next_batch(gen_list[i*batch_size:i*batch_size + batch_size], 1536)
        sequences_dis = next_batch(dis_list[i*batch_size:i*batch_size + batch_size], 3072)
        print(start_data_gen.shape)
        print("got data")
        encoder_loss = encoder_decoder.train_on_batch([sequences_dis,start_data_gen], [sequences_dis, start_data_gen])
        print("step 1"),
        total_fake_data = generator.predict(start_data_gen)
        d_loss_fake = discriminator.train_on_batch(total_fake_data, tf.zeros(batch_size))
        d_loss_real = real_discriminator.train_on_batch(sequences_dis, tf.ones(batch_size)*.9)
        print(real_discriminator.predict(sequences_dis))
        print(d_loss_real)
        print("step 2")
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        g_loss = combined.train_on_batch(start_data_gen, tf.ones(batch_size))
        g_test = decoder_model.predict(generator.predict(start_data_gen))
        print(g_test)
        print("batch {} completed! encoder loss: {} discriminator loss: {} generator_loss: {}".format(i, encoder_loss[0],d_loss,g_loss))
        if i % 10 == 0:
            print("10 batches completed")
            single_start_data = next_batch([random.randint(0,num_total_data)], 1536)
            sample_new_data = generator.predict(single_start_data)
            with open("./test_results/test1.pickle", "wb+") as file:
                pickle.dump(sample_new_data, file)
                              
                        

getting data


W0816 14:39:19.309401 4321366912 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


(100, 1536, 88)
got data
step 1


W0816 14:39:30.428936 4321366912 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
W0816 14:39:51.725167 4321366912 training.py:2197] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


[[0.4877426 ]
 [0.48261848]
 [0.49350822]
 [0.49586377]
 [0.509082  ]
 [0.49852195]
 [0.4973284 ]
 [0.4812055 ]
 [0.48764992]
 [0.49448323]
 [0.50399435]
 [0.48919496]
 [0.49481732]
 [0.49160644]
 [0.4949011 ]
 [0.49997577]
 [0.4856197 ]
 [0.49899596]
 [0.49198276]
 [0.4941563 ]
 [0.4940291 ]
 [0.49354017]
 [0.50033164]
 [0.49112967]
 [0.49514493]
 [0.49254295]
 [0.5075601 ]
 [0.49006552]
 [0.49428582]
 [0.49747884]
 [0.50362873]
 [0.47807267]
 [0.48466054]
 [0.48814598]
 [0.5054621 ]
 [0.4933034 ]
 [0.4940388 ]
 [0.4983763 ]
 [0.48846254]
 [0.4928892 ]
 [0.49533808]
 [0.48586705]
 [0.4922794 ]
 [0.49203762]
 [0.4977969 ]
 [0.49033204]
 [0.49584946]
 [0.49275547]
 [0.49725142]
 [0.51684517]
 [0.5032636 ]
 [0.49342272]
 [0.49565023]
 [0.49372804]
 [0.50687474]
 [0.5005761 ]
 [0.49661192]
 [0.4999811 ]
 [0.49273446]
 [0.5050194 ]
 [0.4924851 ]
 [0.49604905]
 [0.49401045]
 [0.502818  ]
 [0.4959592 ]
 [0.4917011 ]
 [0.49071386]
 [0.4903041 ]
 [0.4692888 ]
 [0.49794486]
 [0.4881182 ]
 [0.49

(100, 1536, 88)
got data
step 1


KeyboardInterrupt: 